In [ ]:
import torch
from torch import nn
from torch.utils.data.sampler import SubsetRandomSampler
from torch.nn import functional as F

from torchvision.models import resnet18

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class ResNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.resnet11 = resnet18(progress=False).eval()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        #x = self.transforms(x)
        y_pred = self.resnet11(x)
        return F.adaptive_avg_pool1d(y_pred.unsqueeze(1), output_size=1)

In [ ]:
import numpy
import os

class GroundDataset(torch.utils.data.IterableDataset):

    def __init__(self, iterator):
        super(GroundDataset, self).__init__()
        self._iterator = iterator
        self._setup = False

    def _setup_iterator(self):
        self._iterator = self._iterator()

    def __iter__(self):
        if self._setup is False:
            self._setup_iterator()
            self._setup = True
        for x in self._iterator:
            yield x
            
def getItr(directory):

    def __iter__():
        for ds in os.listdir(directory):
            ds_name = os.path.join(directory,ds)
            data = None
            labels = None
            for f in os.listdir(ds_name):
                filename = os.path.join(ds_name,f)  
                if not os.path.isdir(filename):
                    if filename.find("data") > -1:
                        data = numpy.nan_to_num(numpy.load(filename),posinf=0,neginf=0)
                    else:   
                        if filename.find("labels") > -1:
                            labels = numpy.nan_to_num(numpy.load(filename),posinf=0,neginf=0)
                            print (filename,labels.mean())
            #print(data.shape)        
            for i in range(data.shape[0]):  
                yield numpy.transpose(data[i].astype(numpy.float64),(2,1,0)),numpy.asarray([labels[i].flatten().astype(numpy.float64).mean()]).reshape((1,1))
    return __iter__              
            

In [ ]:
learning_rate = 0.000005

model = ResNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [ ]:
num_epochs = 10
model_save_path = "."

iter_ = getItr("processed/train")
train_ds =  GroundDataset(iter_)
iter_ = getItr("processed/test")
test_ds = GroundDataset(iter_)
iter_ = getItr("processed/outumn")
val_ds = GroundDataset(iter_)

train_loss = nn.MSELoss()
dataloader =  torch.utils.data.DataLoader(train_ds, num_workers=0, batch_size = 64, shuffle=False)
valloader = torch.utils.data.DataLoader(val_ds, num_workers=0, batch_size = 64, shuffle=False)
testloader = torch.utils.data.DataLoader(test_ds, num_workers=0, batch_size = 64, shuffle=False)

In [ ]:

total_step = 898#len(train_loader)

for epoch in range(num_epochs):
    for i, (data, labels) in enumerate(dataloader):  
        # Move tensors to the configured device
        data = data.float() 
        data = data.to(device)
        labels = labels.float()
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(data)
        loss = train_loss(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    # Validation
    with torch.no_grad():
        mse = []
        
        for data, labels in valloader:
            data = data.float()
            data = data.to(device)
            labels = labels.float()
            labels = labels.to(device)
            outputs = model(data)
            predicted = outputs.data
            mse.append(float(train_loss(labels.data,predicted))) 
            del data, labels, outputs
    
        print('MSE of the network on the {} validation samples: {} %'.format(5000, numpy.asarray(mse).mean())) 

In [ ]:
with torch.no_grad():
    mse = []

    for data, labels in valloader:
        data = data.float()
        data = data.to(device)
        labels = labels.float()
        labels = labels.to(device)
        outputs = model(data)
        predicted = outputs.data
        mse.append(float(train_loss(labels.data,predicted))) 
        del data, labels, outputs

    print('MSE of the network on the {} test samples: {} %'.format(5000, numpy.asarray(mse).mean())) 

In [ ]:
with torch.no_grad():
    mse = []

    for data, labels in valloader:
        data = data.float()
        data = data.to(device)
        labels = labels.float()
        labels = labels.to(device)
        outputs = model(data)
        predicted = outputs.data
        mse.append(float(train_loss(labels.data,predicted))) 
        del data, labels, outputs

    print('MSE of the network on the {} test samples: {} %'.format(5000, numpy.asarray(mse).mean())) 

In [ ]:
with torch.no_grad():
    mse = []

    for data, labels in valloader:
        data = data.float()
        data = data.to(device)
        labels = labels.float()
        labels = labels.to(device)
        outputs = model(data)
        predicted = outputs.data
        mse.append(float(train_loss(labels.data,predicted))) 
        del data, labels, outputs

    print('MSE of the network on the {} test samples: {} %'.format(5000, numpy.asarray(mse).mean())) 

In [ ]:
for d in dataloader:
    print (d)

In [ ]:
!top